### Descripción de la base de datos

La base de datos seleccionada para este proyecto fue la base [The Extended Yale Face Dataset B](http://vision.ucsd.edu/~iskwak/ExtYaleDatabase/ExtYaleB.html) , en específico las versión recortada utilizada en el artículo [Acquiring Linear Subspaces for Face Recognition under Variable Lighting](http://vision.ucsd.edu/~leekc/papers/9pltsIEEE.pdf). Estas versiones recortadas resultan mejores para este tipo de algoritmos pues dentro de la foto sólo aparece la región entre las orejas y de la frente a la barbilla, así evitamos generar ruido innecesario producido por el cabello, ropa o joyería utilizada por la persona a la que se le toma la foto.
 
La manera en que se recolectaron las fotos de este dataset fue con la ayuda de un domo geodésico que tenía varias lámparas alrededor programadas para activarse en distintos momentos en un intervalo de 2 segundos para generar 64 distintas configuraciones de iluminación. En el centro de este domo se sentaba a la persona que se iba a fotografiar y al activarse la secuencia de iluminación se tomaron 64 fotografías de cada persona. Este procedimiento se repitió para 38 personas.

Cada fotografía en su versión recortada tiene dimensiones de 168 x 192 pixeles lo que, una vez vectorizada la imagen, resulta en un vector con 32,256 entradas. Multiplicando esto por las 64 fotos de las 38 personas, obtenemos una matriz con aproximadamente 78 millones de entradas.

### Algoritmo de Eigenfaces

El problema de las eigenfaces es un problema relativamente viejo, tuvo sus inicios en 1987 con Sirovich y Kirby en su artículo [Low-dimensional procedure for the characterization of human faces](http://pdfs.semanticscholar.org/2a62/d0cca2fabf1d6f6ee15e4c14cef415b657d1.pdf) pero no fue si no hasta 1991 que se estudió su aplicación en reconocimiento facial automatizado con Turk y Pentland en su artículo [Eigenfaces for Recognition](https://www.mitpressjournals.org/doi/pdf/10.1162/jocn.1991.3.1.71).

En este caso, nuestro conjunto de datos tiene 38 personas, tomamos 36 de estas para nuestro conjunto de entrenamiento y resguardamos las 2 restantes para el conjunto de prueba.

Antes de comenzar a trabajar, y como se mencionó en la sección anterior, es necesario pasar cada una de las imágenes a un vector columna y colocar cada uno de estos en una matriz. Posteriormente, obtenemos el vector columna promedio de esta matriz y se lo restamos a todos los vectores para así centrar nuestros datos.

![title](../images/matriz-x.png)

Así, el calcular la SVD de la matriz de fotos centrada se es realmente un problema de cálculo de PCA. Una vez calculada la SVD y tomando en cuenta la notación usual, los vectores columna de la matriz U representan una base ortonormal del del espacio de eigenfaces.

Por las propiedades de la SVD, los vectores columna de $U$ tienen la misma dimensión que los vectores columna de $X$. Por lo tanto, si se obtuvieron los vectores columna de $X$ a partir de una imagen, entonces lo vectores columna de $U$ también pueden reacomodarse para formar una imagen de una cara, estos vectores son conocidos como eigenfaces.

![title](../images/matriz-u.png)

Como en este caso, se tienen 64 imágenes de 36 personas, entonces se tienen alrededor de 23 mil eigenfaces. En este espacio de eigenfaces, se pueden representar cada una de las caras como una combinación lineal de las eigenfaces. Cada cara tiene una representación única en este espacio.

Es importante notar que este espacio no sirve únicamente para representar aquellas caras con las que fue construido si no que también podemos proyectar una cara nueva en este espacio y encontrar su representación ahí. Sin embargo, representar una cara nueva como una combinación lineal de 23 mil vectores columna no es práctico, por lo que podemos hacer entonces una aproximación en en una dimensión más pequeña donde sólo tomemos un subconjunto de eigenfaces, las r eigenfaces correspondientes a los r valores singulares más grandes.

La razón por la que podemos hacer una aproximación a cada cara con un subconjunto de $r$ eigenfaces es por que podemos aproximar X como:
$$ \hat{X} \approx U_r \Sigma_r V_r^T$$
Entonces podemos definir un vector $\alpha$:
$$ \alpha = U_r^T x$$
como el vector que contiene la información de que tanto se necesita de cada eigenface para reconstruir a $x$ (i.e. combinación lineal de eigenfaces). Donde $x$ es el vector columna que representa a la nueva cara. Es este espacio cada $\alpha$ es único para cada cara. Entonces una aproximación de $x$ tomando únicamente $r$ eigenfaces sería:
$$\hat{x} = U_r U_r^T x$$

Esta es una compresión muy grande pues para guardar una foto en vez de utilizar los 168 x 192 pixeles únicamente necesitariamos un vector de tamaño $r$ con la información de la combinación lineal de eigenfaces para representar todas las caras del mundo. Por suspuesto que para que esta forma de compresión tuviera éxito, deberíamos de ponernos todos de acuerdo en que base de datos utilizar para hacer esta representación.

Así, más adelante cuando se presente el código nos enfocaremos en buscar el subconjunto de $r$ eigenfaces que nos den una buena representación de una cara fuera del conjunto de entrenamiento para demostrar la gran reducción de dimensionalidad que se puede producir.

Para construir la nueva cara, primero es necesario hallar su representación vectorial y antes de aplicar el algoritmo de aproximación descrito anteriormente, sumar la cara promedio.

![title](../images/recon37.png)

Otra aplicación de esta aproximación $\alpha$ es que puede ser usada para clasificación de imágenes. En vez de realizar la clasificación de una cara $x$ en un espacio de muchas dimensiones se puede hacer la proyección de $x$ en el espacio generado por el subconjunto de $r$ eigenfaces. Más adelante, observaremos que las imágenes de la misma persona tienden a aglomerarse, por lo que esto puede usarse como una forma sencilla de reconocimiento de caras.
 
Esta aglomeración puede visualizarse cuando se grafica de acuerdo con los componentes principales (eigenfaces). En general, cuando se habla de rostros, los primeros componentes principales tienden a contener cierta información que es la misma para todas las caras, como que se tienen ojos, nariz y boca. La información más específica como la forma de la nariz, la anchura de la boca o el tamaño de la barbilla se almacena en el resto de las eigenfaces.

**aqui van más cosas**

## Código
